In [ ]:
#@title 1. Замена лица на видео
from google.colab import drive
import os

#@markdown   Использовать гугл диск:
use_gdrive = False #@param {type:"boolean"}
#@markdown   Укажите путь до файла изображения с новым лицом:
use_this_face = "" #@param {type:"string"}
#@markdown   Укажите путь до файла с видео или фото, на котором будем делать замену:
replace_this_face = "" #@param {type:"string"}
#@markdown   Сохранить папку с изображениями раскадровки:
keep_frames_directory = False #@param {type:"boolean"}
#@markdown   Поменять лица у всех на видео
all_faces = False #@param {type:"boolean"}

_kf_dir = ""
_all_faces = ""
_face = f'-f {use_this_face}'
_target = f'-t {replace_this_face}'

if use_gdrive:
    drive.mount('/content/drive')

if keep_frames_directory:
    _kf_dir = "--keep-frames"
else:
    _kf_dir = ""

if all_faces:
    _all_faces = "--all-faces"
else:
    _all_faces = ""

if not os.path.exists('/content/roop/run.py'):
    %cd /content

    if os.path.exists('/content/roop'):
        !mv /content/roop /content/roop-temp

    !git clone https://github.com/s0md3v/roop

    if os.path.exists('/content/roop-temp'):
        !cp -rf /content/roop-temp/* -t /content/roop && rm -r /content/roop-temp
    
    %cd /content/roop
    !pip install -q -r requirements.txt
    !pip uninstall onnxruntime onnxruntime-gpu
    !pip install torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/cu118
    !pip install onnxruntime-gpu

    if not os.path.exists('/content/roop/inswapper_128.onnx'):
        !wget -O /content/roop/inswapper_128.onnx 'https://civitai.com/api/download/models/85159?type=Model&format=Other&size=full&fp=fp32'
if not use_this_face == "" and not replace_this_face == "":
    %cd /content/roop
    !python3 run.py $_face $_target $_kf_dir $_all_faces --gpu-vendor nvidia
else:
    print('Поля use_this_face и replace_this_face обязательны!')